In [1]:
import re
from dateparser.search import search_dates
import os
import pandas as pd
import import_ipynb
from Tesseract import *

importing Jupyter notebook from Tesseract.ipynb


In [2]:
DataDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")

In [3]:
def get_invoice_nums(all_words):
    inv_nums = []
    invoice_no_re = r'^[0-9a-zA-Z-:/]+$'
    for word in all_words:
        if not re.search('\d', word['text']):
            continue
        result = re.findall(invoice_no_re,word['text'])
        if result:
            inv_nums.append({
                'text': word['text'],
                'x1': word['left'],
                'y1': word['top'],
                'x2': word['left'] + word['width'],
                'y2': word['top'] + word['height']
            })

    return inv_nums

In [4]:
def get_amounts(all_words):
    amounts = []
    amount_re = r"^\$?\d+(,\d{3})*(\.\d+)?$"
    for word in all_words:
        if not re.search(amount_re, word['text']):
            continue
        if len(word)<3:
            continue
        result = re.findall(amount_re,word['text'])
        if result:
            amounts.append({
                'text': word['text'],
                'x1': word['left'],
                'y1': word['top'],
                'x2': word['left'] + word['width'],
                'y2': word['top'] + word['height']
            })

    return amounts

In [5]:
def get_dates(all_text, all_words):
    dates, all_dates = [], []
    indices = []
    matches = search_dates(all_text)

    
    for match in matches:
        text = match[0]

        token_length = len(text.split(' '))
        idx = all_text.find(match[0])
        text_len = len(text)
        index = len(all_text[:idx].strip().split(' '))

        replaced_text = ' '.join(['*'*len(i) for i in text.split(' ')])

        indices.append(list(range(index, index + token_length)))

        index += token_length
        all_text = all_text[:idx + text_len].replace(text, replaced_text) + all_text[idx + text_len:]
    
    for date_indices in indices:
        date = ''
        left, top, right, bottom = [], [], [], []
        
        for i in date_indices:
            if i<len(all_words):
                date += ' ' + all_words[i]['text']
                left.append(all_words[i]['left'])
                top.append(all_words[i]['top'])
                right.append(all_words[i]['left'] + all_words[i]['width'])
                bottom.append(all_words[i]['top'] + all_words[i]['height'])
        all_dates.append({
            'text': date.strip(),
            'x1': min(left),
            'y1': min(top),
            'x2': max(right),
            'y2': max(bottom)
        })

    return all_dates


In [6]:
def get_candidates(file):
        image = cv2.imread(file)
        data = ocr(file)
        data["left"]/=image.shape[1]
        data["top"]/=image.shape[0]
        data["width"]/=image.shape[1]
        data["height"]/=image.shape[0]
        data["X"]/=image.shape[1]
        data["Y"]/=image.shape[0]

        all_words = []
        for idx in data.index:
            word = data["text"][idx]
            if word.strip() != "":
                all_words.append({
                    'text': data['text'][idx],
                    'left': data['left'][idx],
                    'top': data['top'][idx],
                    'width': data['width'][idx],
                    'height': data['height'][idx]})
        text = ' '.join([word['text'].strip() for word in all_words])

        invoice_date_candidates = get_dates(text,all_words)
    
        total_amount_candidates = get_amounts(all_words)
    
        invoice_no_candidates = get_invoice_nums(all_words)

        candidate_data = {
            'invoice_no': invoice_no_candidates,
            'invoice_date': invoice_date_candidates,
            'total_amount': total_amount_candidates
        }
        Dataframes = []
        for key in candidate_data.keys():
            df = pd.DataFrame(candidate_data[key])
            df["Class"] = key
            Dataframes.append(df)
        return data, Dataframes

In [7]:
D, DF = get_candidates("1.jpeg")

In [8]:
D

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,x1,y1,x2,y2,X,Y
4,5,1,1,1,1,1,0.392900,0.026826,0.033885,0.009989,95.928436,Tax,974,94,1058,129,0.409843,0.031821
5,5,1,1,1,1,2,0.434449,0.026256,0.067366,0.010559,95.960678,Invoice,1077,92,1244,129,0.468132,0.031535
9,5,1,2,1,1,1,0.060508,0.054509,0.881404,0.000285,95.000000,,150,191,2335,192,0.501210,0.054652
13,5,1,3,1,1,1,0.067366,0.063071,0.121017,0.011416,92.071617,BANDHAVI,167,221,467,261,0.127874,0.068779
14,5,1,3,1,1,2,0.198064,0.062785,0.163372,0.011986,96.154831,ENTERPRISES,491,220,896,262,0.279750,0.068779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,5,1,50,1,1,2,0.351755,0.947203,0.009278,0.007991,96.532051,is,872,3319,895,3347,0.356394,0.951199
400,5,1,50,1,1,3,0.372328,0.949201,0.002017,0.005708,93.246674,a,923,3326,928,3346,0.373336,0.952055
401,5,1,50,1,1,4,0.379185,0.947203,0.141186,0.009989,91.934959,Computer-Generated,940,3319,1290,3354,0.449778,0.952197
402,5,1,50,1,1,5,0.526825,0.947203,0.045986,0.007705,95.395515,Invoice,1306,3319,1420,3346,0.549818,0.951056
